In [1]:
import pymongo as pm
import pandas as pd
from datetime import datetime
# Create the client
db_client = pm.MongoClient('####')
db = db_client.mygig

In [3]:
# Ограничиваем период
start = datetime(2023, 9, 1)
finish = datetime(2023, 9, 16)

In [4]:
# Подгрузка данных
jobs = pd.DataFrame(db['jobs'].find({'start': {'$gte': start, '$lt': finish}}))
workers = pd.DataFrame(db['workers'].find(
    {'_id': {'$in': list(jobs['worker'])}}))
users = pd.DataFrame(db['users'].find({'_id': {'$in': list(workers['user'])}}))
workplaces = pd.DataFrame(db['workplaces'].find(
    {'_id': {'$in': list(jobs['workplace'])}}))

In [5]:
# Формирование ДС
df2 = jobs[(jobs['status'] == 'confirmed') &
           (jobs['client'].astype(str).isin(['####']))][
    ['workplace', 'start', 'worker', 'start_time', 'finish_time', 'hours', 'custom_name']].merge(
    workers[['_id', 'user']],
    left_on='worker', right_on='_id', how='left').merge(
    users[['_id', 'firstname', 'lastname', 'middlename']],
    left_on='user', right_on='_id', how='left').merge(
    workplaces[['_id', 'title']],
    left_on='workplace', right_on='_id', how='left')
df2['day'] = df2['start'].dt.day
df2['ФИО'] = df2['lastname'].fillna('').astype(
    str)+' '+df2['firstname'].fillna('').astype(str)+' '+df2['middlename'].fillna('').astype(str)
df2['job'] = df2['start_time'].fillna('').astype(
    str)+'-'+df2['finish_time'].fillna('').astype(str)+' ('+df2['hours'].fillna(0).astype(str)+')'

In [7]:
#Итоговая форма
pd.concat([df2.pivot(index=['title', 'ФИО', 'custom_name'],
          columns='day',
          values='job'),
           df2.pivot_table(index=['title', 'ФИО', 'custom_name'],
                           values='hours',
                           aggfunc='sum')], axis=1).to_excel(
    "####")